In [1]:
import pandas as pd

In [2]:
pd.__version__

'2.2.3'

In [4]:
!pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 57.3 MB/s eta 0:00:000m eta 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")

In [6]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [7]:
import sklearn
sklearn.__version__

'1.6.1'

# Homework

In [2]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    # df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [3]:
df_jan = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet") # "Yellow Taxi Trip Records"
df_feb = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [15]:
df_jan.shape

(3066766, 20)

In [16]:
df_jan["duration"].std()

np.float64(42.59435124195458)

In [4]:
df_jan = df_jan[(df_jan.duration >= 1) & (df_jan.duration <= 60)]
df_feb = df_feb[(df_feb.duration >= 1) & (df_feb.duration <= 60)]

df_jan.shape[0]/3066766

0.9812202822125979

In [5]:
df_jan.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [13]:
from sklearn.feature_extraction import DictVectorizer

categorical = ["PULocationID", "DOLocationID"]

df_jan = df_jan[(df_jan["PULocationID"] != "199")]
df_jan[categorical] = df_jan[categorical].astype(str)
jan_dicts = df_jan[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(jan_dicts)

In [22]:
X_train # before filter

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018346 stored elements and shape (3009173, 515)>

In [14]:
X_train # after filter

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6018342 stored elements and shape (3009171, 514)>

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

target = 'duration'
y_train = df_jan[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.6492644467915945

In [10]:
set(df_jan["PULocationID"]).difference(set(df_feb["DOLocationID"]))

{'199'}

In [10]:
set(df_feb["PULocationID"]).difference(set(df_jan["PULocationID"]))

{'105', '204', '59'}

In [11]:
set(df_feb["DOLocationID"]).difference(set(df_jan["DOLocationID"]))

{'105'}

In [16]:
df_feb[categorical] = df_feb[categorical].astype(str)
feb_dicts = df_feb[categorical].to_dict(orient='records')
X_test = dv.fit_transform(feb_dicts)
y_test = df_feb[target].values

y_pred = lr.predict(X_test)

root_mean_squared_error(y_test, y_pred)

16.33514297133926